# Feature Matching

This notebook will guide you through the process of finding corresponding features between two images. This is a fundamental task in computer vision with applications in 3D reconstruction, object recognition, loop closure, and more.

We will cover the following steps:
2.  **Feature Detection and Extraction**: Use the SIFT (Scale-Invariant Feature Transform) algorithm to detect keypoints and compute their descriptors.
3.  **Feature Matching**: Match the detected features between the two images.
4.  **Geometric Model Estimation**: Use RANSAC to estimate a geometric transformation (Homography) from the matches.
5.  **Visualization**: Visualize the results, including matched features, inliers/outliers, and warped images or epilines.

In [ ]:
# JUST RUN THIS CELL

%load_ext autoreload
%autoreload 2

%matplotlib inline

import cv2 # opencv is a computer vision library
import numpy as np # numpy is a library for numerical computing
from matplotlib import pyplot as plt # matplotlib is a plotting library
import pathlib
import tqdm
from enum import Enum

import sys
import os
sys.path.append('../../')

import utils
from utils import show_image, show_images, warp_perspective, show_inlier_matches

cwd = os.getcwd()
feature_1_block_dir = pathlib.Path(os.getcwd()).parent
images_path = feature_1_block_dir / 'images'

In [ ]:
# JUST RUN THIS CELL

utils.CV2_MAX_IMAGE_HEIGHT = 1080/2  # set a maximum height for cv2 image visualization
utils.SHOW_IMAGE_BACKEND = utils.Backend.PLT  # set the default backend for visualization
utils.DRAW_MATCH_THICKNESS = 2

Let's load two images containing the same object. We can apply some transformations to the images to make the feature matching more difficult and interesting.

*But attention!* If you modify the images too much, it will not be possible anymore to robustly recognize the same common features in the two images. Suggestion: first complete the exercise without touching these transform parameters, and come back later to this cell to experiment with them.

In [ ]:
IMAGE_NAME_1 = 'harry_potter_cover.jpg'
IMAGE_NAME_2 = 'harry_potter_library.jpg'

### PLAY WITH THESE PARAMETERS TO SEE DIFFERENT RESULTS
RESIZE_FACTOR_1 = 0.5
RESIZE_FACTOR_2 = 1.0

BRIGHTNESS_SCALE_ALPHA_1 = 0.8
BRIGHTNESS_OFFSET_BETA_1 = 0

BRIGHTNESS_SCALE_ALPHA_2 = 1.0
BRIGHTNESS_OFFSET_BETA_2 = 0

SCALE_1 = 1.2
TRANSLATE_X_1 = 10.0
TRANSLATE_Y_1 = 30.0
SHEAR_X_1 = 100.0
SHEAR_Y_1 = -200.0
ROTATION_1 = 20.0

SCALE_2 = 1.0
TRANSLATE_X_2 = 0.0
TRANSLATE_Y_2 = 0.0
SHEAR_X_2 = 0.0
SHEAR_Y_2 = 0.0
ROTATION_2 = 0.0
#####################################

image_path_1 = images_path / IMAGE_NAME_1
image_path_2 = images_path / IMAGE_NAME_2

input_image_1 = cv2.imread(str(image_path_1))
input_image_1 = cv2.resize(input_image_1, (0,0), fx=RESIZE_FACTOR_1, fy=RESIZE_FACTOR_1)
input_image_1 = cv2.convertScaleAbs(input_image_1, alpha=BRIGHTNESS_SCALE_ALPHA_1, beta=BRIGHTNESS_OFFSET_BETA_1)
input_image_1 = warp_perspective(input_image_1, scale=SCALE_1, tx=TRANSLATE_X_1, ty=TRANSLATE_Y_1,
                                 shear_x=SHEAR_X_1, shear_y=SHEAR_Y_1, rot_deg=ROTATION_1)
mask_image_1 = np.ones_like(input_image_1, dtype=np.uint8)
mask_image_1 = warp_perspective(mask_image_1, scale=SCALE_1, tx=TRANSLATE_X_1, ty=TRANSLATE_Y_1,
								 shear_x=SHEAR_X_1, shear_y=SHEAR_Y_1, rot_deg=ROTATION_1)


input_image_2 = cv2.imread(str(image_path_2))
input_image_2 = cv2.resize(input_image_2, (0,0), fx=RESIZE_FACTOR_2, fy=RESIZE_FACTOR_2)
input_image_2 = cv2.convertScaleAbs(input_image_2, alpha=BRIGHTNESS_SCALE_ALPHA_2, beta=BRIGHTNESS_OFFSET_BETA_2)
input_image_2 = warp_perspective(input_image_2, scale=SCALE_2, tx=TRANSLATE_X_2, ty=TRANSLATE_Y_2,
                                 shear_x=SHEAR_X_2, shear_y=SHEAR_Y_2, rot_deg=ROTATION_2)

show_images([input_image_1, input_image_2], ["Input Image 1", "Input Image 2"])

### SIFT keypoint detection and descriptor extraction

This cell detects SIFT keypoints and computes their descriptors for each image. The resulting keypoints and descriptors are used for matching.

In [ ]:
### PLAY WITH THESE PARAMETERS TO REFINE THE FEATURE DETECTION
SIFT_N_FEATURES = None
SIFT_N_OCTAVE_LAYERS = 3
SIFT_CONTRAST_THRESHOLD = 0.04
SIFT_EDGE_THRESHOLD = 10
SIFT_SIGMA = 1.6
#########################################

SIFT_RICH_VISUALIZATION = True
SIFT_KEYPOINT_COLOR = None # (B,G,R) color tuple or None for random color per keypoint

sift_extractor = cv2.SIFT_create(
	nfeatures=SIFT_N_FEATURES,
	nOctaveLayers=SIFT_N_OCTAVE_LAYERS,
	contrastThreshold=SIFT_CONTRAST_THRESHOLD,
	edgeThreshold=SIFT_EDGE_THRESHOLD,
	sigma=SIFT_SIGMA
)

# extract keypoints
sift_keypoints_1, sift_descriptors_1 = sift_extractor.detectAndCompute(input_image_1, None)
print(f"Detected {len(sift_keypoints_1)} SIFT keypoints in image 1")

# compute the descriptors at the detected keypoints
sift_keypoints_2, sift_descriptors_2 = sift_extractor.detectAndCompute(input_image_2, None)
print(f"Detected {len(sift_keypoints_2)} SIFT keypoints in image 2")

sift_img = input_image_1.copy()
sift_viz_flag = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS if SIFT_RICH_VISUALIZATION else 0
cv2.drawKeypoints(sift_img, sift_keypoints_1, sift_img, color=SIFT_KEYPOINT_COLOR, flags=sift_viz_flag)

sift_img_2 = input_image_2.copy()
cv2.drawKeypoints(sift_img_2, sift_keypoints_2, sift_img_2, color=SIFT_KEYPOINT_COLOR, flags=sift_viz_flag)

show_images([sift_img, sift_img_2], ["SIFT Keypoints Image 1", "SIFT Keypoints Image 2"])

## Descriptor matching

This cell matches descriptors between the two images using the configured matcher (Brute-Force or FLANN) and filters matches using ratio and distance thresholds.

Your task:

The next three code 

In [ ]:
### PLAY WITH THESE PARAMETERS TO REFINE THE MATCHING
K = 2  # number of nearest neighbors to find (K>1 useful to apply Lowe's ratio test)
DISTANCE_RATIO_THRESHOLD = 0.9  # Lowe's ratio test threshold to filter ambiguous matches
DISTANCE_ABSOLUTE_THRESHOLD = 500  # absolute distance threshold to filter outliers
#########################################

#TODO: implement Lowe's ratio test function to filter ambiguous matches
def TODO_Lowe_ratio_distance_filtering(
		matches: list[tuple[cv2.DMatch, cv2.DMatch]],
		distance_ratio_threshold: float
	) -> list[cv2.DMatch]:
	# Apply Lowe's ratio test on the first two nearest neighbors (they are returned sorted by distance)
	good_matches = []
	for m1, m2, *_ in matches:
		if True: # TODO: implement the actual test using distance_ratio_threshold
			good_matches.append(m1)
	return good_matches

###########################################

def matching(sift_descriptors_1, sift_descriptors_2, K, DISTANCE_RATIO_THRESHOLD, DISTANCE_ABSOLUTE_THRESHOLD):
	flann = cv2.FlannBasedMatcher()
	matches = flann.knnMatch(sift_descriptors_1, sift_descriptors_2, k=K)

	# Apply Lowe's ratio test to filter good matches
	if K == 1:
		good_matches = [m[0] for m in matches if len(m) > 0]  # if K=1, all matches are considered good
	else:
		good_matches = TODO_Lowe_ratio_distance_filtering(matches, DISTANCE_RATIO_THRESHOLD)

	# Further filter matches based on absolute distance threshold
	good_matches = [m for m in good_matches if m.distance < DISTANCE_ABSOLUTE_THRESHOLD]

	return good_matches

good_matches = matching(sift_descriptors_1, sift_descriptors_2,
	K, DISTANCE_RATIO_THRESHOLD, DISTANCE_ABSOLUTE_THRESHOLD)

# Draw matches
SIFT_matches = cv2.drawMatches(input_image_1, sift_keypoints_1, input_image_2, sift_keypoints_2,
	good_matches, None, matchesThickness=utils.DRAW_MATCH_THICKNESS, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show_image(SIFT_matches, f"{len(good_matches)} SIFT Matches")

### Robust geometry estimation (RANSAC)

Now let's repeat the previous parth, but this time we will use very bad matching parameters to obtain a lot of outlier matches, and we will use RANSAC applied to an homography geometric model to filter them. You will see that RANSAC can be very good in rejecting outliers, even when the majority of the matches are outliers.

Your task:
1) Fix the `TODO_find_homography_ransac` function. Check the [OpenCV documentation for the findHomography function](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga4abc2ece9fab9398f2e560d53c8c9780).
2) Play with the parameters to improve and refine the outlier rejection result.

In [ ]:
### PLAY WITH THESE PARAMETERS TO REFINE THE MATCHING
RANSAC_REPROJ_THRESH = 10.0
RANSAC_CONFIDENCE = 0.9
RANSAC_MAX_ITERS = 30
########################

# TODO: Fix the Homography estimation with RANSAC to filter out outlier matches
def TODO_find_homography_ransac(
		pts1: np.ndarray,
		pts2: np.ndarray,
		reproj_threshold: float,
		max_iters: int,
		confidence: float
	) -> tuple[np.ndarray, np.ndarray]:
	# Estimate Homography using RANSAC to filter out outlier matches
	# Check https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga4abc2ece9fab9398f2e560d53c8c9780
	H = None
	mask = None
	# H, mask = cv2.findHomography(...)

	if H is None:
		raise RuntimeError("Homography estimation failed. Try to run again (Ransac is random!), or adjust parameters.")

	return H, mask

########################################

# previous function to find matches. Let's use very bad parameters to have many outliers
bad_K = 1
bad_distance_ratio_threshold = 1.0
bad_distance_absolute_threshold = 1000
good_matches = matching(sift_descriptors_1, sift_descriptors_2,
	bad_K, bad_distance_ratio_threshold, bad_distance_absolute_threshold)

# Draw matches
SIFT_matches = cv2.drawMatches(input_image_1, sift_keypoints_1, input_image_2, sift_keypoints_2,
	good_matches, None, matchesThickness=utils.DRAW_MATCH_THICKNESS, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show_image(SIFT_matches, f"{len(good_matches)} SIFT Matches")


# NEW PART: use RANSAC to estimate a Homography and filter out outlier matches

if len(good_matches) == 0:
	raise RuntimeError("No matches to estimate transformation.")

pts1 = np.float32([sift_keypoints_1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
pts2 = np.float32([sift_keypoints_2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

H, mask = TODO_find_homography_ransac(pts1, pts2, RANSAC_REPROJ_THRESH, RANSAC_MAX_ITERS, RANSAC_CONFIDENCE)

inlier_mask = (mask.ravel() == 1)

print(f"Homography Matrix estimated with {int(np.count_nonzero(inlier_mask))}/{len(good_matches)} inliers:\n{H}")

# visualize matches: green = inlier, red = outlier
show_inlier_matches(input_image_1, sift_keypoints_1, input_image_2, sift_keypoints_2, good_matches, inlier_mask)

### Warp image

Now we can visualize the quality of the estimated homography matrix. If the first image is correctly warped onto the second image’s viewpoint, it means the keypoint matching and outlier rejection were successful.

In [ ]:
# JUST RUN THIS CELL

if H is None:
    raise RuntimeError("Homography estimation failed. Try to run again (Ransac is random!), or adjust parameters.")

height, width, _ = input_image_2.shape
warped_image_1 = cv2.warpPerspective(input_image_1, H, (width, height))
mask_image_1 = cv2.warpPerspective(mask_image_1.astype(np.float32), H, (width, height)).astype(bool)
show_images([warped_image_1, input_image_2], ["Warped Image 1 using Homography", "Input Image 2"])

merged_image = input_image_2.copy()
merged_image[mask_image_1] = warped_image_1[mask_image_1]
show_image(merged_image, "Merged Image using Homography")

## Conclusion

In this notebook we implemented a feature matching pipeline and explored how local features enable geometric reasoning between images. We have seen how to:

- Match descriptors using nearest-neighbor search (FLANN) and filter ambiguous matches with Lowe’s ratio test and distance thresholds.
- Use RANSAC to robustly estimate a homography and reject outlier correspondences.
- Validate the estimated model by visualizing inlier/outlier matches, warping one image onto the other to inspect alignment.


***********************************************************************
### IMPORTANT OVERVIEW

This simple exercise highlights the broader power of feature detection and matching. **These techniques are not only useful for identifying objects or regions of interest within images, but they also play a crucial role in estimating movement, transformations, and spatial relationships between frames.** By tracking how features shift from one image to another, we can infer camera motion, object motion, scene geometry, and even build consistent representations of the world across multiple views.


In other words, feature correspondences form the foundation for tasks like motion estimation, image stitching, 3D reconstruction, and simultaneous localization and mapping (SLAM). The reliability of the homography you see here is just one example of how these underlying methods enable robust geometric understanding in computer vision systems.
